In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

In [2]:
from liana.method.sp._Misty import MistyData
from liana.method.sp._misty_constructs import genericMistyData, lrMistyData

In [3]:
adata = sc.read_h5ad('liana/tests/data/synthetic.h5ad')

In [4]:
misty = genericMistyData(intra=adata, set_diag=True, bandwidth=10)

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [5]:
misty(n_estimators=20, bypass_intra=False)

In [6]:
misty.uns['target_metrics']

,target,intra_group,env_group,intra.R2,multi.R2,gain.R2,intra,juxta,para
0,ECM,None,None,0.916691,0.935051,0.018360,0.770407,0.138171,0.091422
1,ligA,None,None,0.988682,0.988936,0.000254,0.966251,0.026840,0.006909
2,ligB,None,None,0.931729,0.945597,0.013869,0.794769,0.130086,0.075145
3,ligC,None,None,0.772048,0.867912,0.095864,0.579776,0.241276,0.178948
4,ligD,None,None,0.975031,0.976384,0.001353,0.924838,0.040185,0.034977
5,protE,None,None,0.996984,0.996995,0.000010,1.000000,0.000000,0.000000
6,protF,None,None,0.998878,0.998882,0.000003,0.994822,0.000217,0.004961
7,prodA,None,None,0.991524,0.991535,0.000011,1.000000,0.000000,0.000000
8,prodB,None,None,0.980290,0.980210,-0.000079,1.000000,0.000000,0.000000
9,prodC,None,None,0.954825,0.955203,0.000378,1.000000,0.000000,0.000000


In [ ]:
adata = sc.pp.subsample(adata, n_obs=100, copy=True)

In [ ]:
misty = genericMistyData(adata, bandwidth=20, add_juxta=True, set_diag=True, cutoff=0, coord_type="generic", delaunay=True)

In [ ]:
# Why is this thing so much slower?
misty(alphas=1, 
      bypass_intra=True,
      seed=42,
      n_estimators=100,
      keep_same_predictor=False,
      group_intra_by='cell_type',
      group_env_by='cell_type',
      )

In [ ]:
misty.uns['importances'].sort_values('value', ascending=False)

In [ ]:
misty.uns['target_metrics'].sort_values('multi.R2') #NOTE: this is weird, why is the R2 not bound to 0-1???

In [ ]:
misty.uns['target_metrics']

In [ ]:
misty.uns['importances']

lrMistyData

In [7]:
from liana.testing._sample_anndata import generate_toy_spatial

In [8]:
adata = generate_toy_spatial()

In [9]:
lr_misty = lrMistyData(adata=adata, set_diag=True, bandwidth=10, use_raw=True)

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [10]:
lr_misty['intra'].var_names

Index(['CD4', 'LAG3', 'FPR1', 'CD8A_CD8B', 'CD40', 'ITGB2', 'ITGA4',
       'TNFRSF13B', 'HLA-DPB1', 'TNFRSF17', 'CD74_CXCR4', 'CD63', 'CD33',
       'S1PR5', 'S1PR4', 'CD53'],
      dtype='object')

In [11]:
lr_misty

MuData object with n_obs × n_vars = 700 × 39
  obs:	'bulk_labels', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'louvain'
  2 modalities
    intra:	700 x 16
      obs:	'bulk_labels', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'louvain'
    extra:	700 x 23
      obsm:	'spatial'
      obsp:	'spatial_connectivities'

In [ ]:
lr_misty(bypass_intra=True)

In [ ]:
lr_misty.uns['target_metrics']

In [ ]:
lr_misty.uns['importances'].sort_values('value', ascending=False)

In [ ]:
from liana.resource import select_resource
from liana.method._pipe_utils import prep_check_adata
from liana.method.sp._lr_basis import _add_complexes_to_var
from liana.method.sp._misty_constructs import _make_view
from liana.method.sp._spatial_pipe import spatial_neighbors

In [ ]:
def lrMistyData(adata, 
                resource_name='consensus', 
                resource=None,
                nz_threshold=0.1,
                use_raw = False,
                layer = None,
                spatial_key='spatial',
                bandwidth = 100,
                kernel = 'misty_rbf',
                set_diag = True,
                cutoff = 0.1,
                zoi = 0,
                verbose = False
                ):
    
    if resource is None:
        resource = select_resource(resource_name.lower())
        
    adata = prep_check_adata(adata=adata,
                        use_raw=use_raw,
                        layer=layer,
                        verbose=verbose,
                        groupby=None,
                        min_cells=None,
                        obsm = {spatial_key: adata.obsm[spatial_key]}
                        )
    
    adata = _add_complexes_to_var(adata,
                              np.union1d(resource['receptor'].astype(str),
                                         resource['ligand'].astype(str)
                                        )
                              )
    
    views = dict()
    intra = _make_view(adata=adata[:, resource['receptor'].unique()],
                   nz_threshold=nz_threshold, obs=adata.obs)
    views['intra'] = _add_complexes_to_var(intra, np.unique(resource['receptor']))
    
    connectivity = spatial_neighbors(adata=adata, spatial_key=spatial_key,
                                 bandwidth=bandwidth, kernel=kernel,
                                 set_diag=set_diag, cutoff=cutoff,
                                 zoi=zoi, inplace=False)

    extra = _make_view(adata=adata[:,resource['ligand'].unique()],
                       spatial_key=spatial_key, nz_threshold=nz_threshold,
                       connecitivity=connectivity,
                       obs=pd.DataFrame(index=adata.obs.index))
    views['extra'] = _add_complexes_to_var(extra, np.unique(resource['ligand']))
    
    return MistyData(data=views, obs=intra.obs, spatial_key=spatial_key)

In [ ]:
use_raw = False
layer = None
verbose = True
resource = None
resource_name = 'consensus'
nz_threshold = 0.1

In [ ]:
spatial_key = 'spatial'
bandwidth = 100
kernel = 'misty_rbf'
set_diag = False
cutoff = 0.1
zoi = 0

In [ ]:
if resource is None:
    resource = select_resource(resource_name.lower())

In [ ]:
views = dict()

In [ ]:
adata = prep_check_adata(adata=adata,
                         use_raw=use_raw,
                         layer=layer,
                         verbose=verbose,
                         groupby=None,
                         min_cells=None,
                         obsm = {spatial_key: adata.obsm[spatial_key]}
                         )

In [ ]:
adata = _add_complexes_to_var(adata,
                              np.union1d(resource['receptor'].astype(str),
                                         resource['ligand'].astype(str)
                                        )
                              )

In [ ]:
adata

In [ ]:
# filter_resource
resource = resource[(np.isin(resource.ligand, adata.var_names)) &
                    (np.isin(resource.receptor, adata.var_names))]

In [ ]:
intra = _make_view(adata=adata[:, resource['receptor'].unique()],
                   nz_threshold=nz_threshold, add_obs=True)

views['intra'] = _add_complexes_to_var(intra, np.unique(resource['receptor']))

In [ ]:
connectivity = spatial_neighbors(adata=adata, spatial_key=spatial_key,
                                 bandwidth=bandwidth, kernel=kernel,
                                 set_diag=set_diag, cutoff=cutoff,
                                 zoi=zoi, inplace=False)

extra = _make_view(adata=adata[:,resource['ligand'].unique()], spatial_key=spatial_key,
                   nz_threshold=nz_threshold, connecitivity=connectivity)
views['extra'] = _add_complexes_to_var(extra, np.unique(resource['ligand']))

In [ ]:
intra.var_names

In [ ]:
from mudata import MuData

In [ ]:
lr_misty = MistyData(data=views, obs=intra.obs, spatial_key=spatial_key)

In [ ]:
lr_misty(bypass_intra=False)

In [ ]:
lr_misty.uns['target_metrics'].sort_values('multi.R2')

In [ ]:
lr_misty.uns['importances'].sort_values('value', ascending=False).head(20)